# Composite Types

In [1]:
struct MyVec1
    x
    y
end

In [2]:
v = MyVec1(1, 2)

MyVec1(1, 2)

In [4]:
v.y

2

In [6]:
struct MyVec2
    x::Float64
    y::Float64
end

In [7]:
MyVec2(1, 2)

MyVec2(1.0, 2.0)

# Parameters

In [8]:
struct MyVec3{T}
    x::T
    y::T
end

In [9]:
MyVec3(1.0, 2.0)

MyVec3{Float64}(1.0, 2.0)

In [10]:
v = MyVec3(1, 2)

MyVec3{Int64}(1, 2)

In [11]:
MyVec3(1, 2.0)

MethodError: MethodError: no method matching MyVec3(::Int64, ::Float64)
The type `MyVec3` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  MyVec3(::T, !Matched::T) where T
   @ Main ~/dev/dmu/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:2


In [12]:
v.x += 2

ErrorException: setfield!: immutable struct of type MyVec3 cannot be changed

In [13]:
struct YourVec{T <: Number}
    x::T
    y::T
    z::String
end

In [14]:
YourVec(1,2,"three")

YourVec{Int64}(1, 2, "three")

In [15]:
struct YourVec2{T1, T2}
    x::T1
    y::T1
    z::T2
end

In [16]:
YourVec2{Float32, ComplexF64}(1, 2, 3)

YourVec2{Float32, ComplexF64}(1.0f0, 2.0f0, 3.0 + 0.0im)

# Mutability

In [17]:
mutable struct MyVec4{T}
    x::T
    y::T
end

In [18]:
v = MyVec4(1, 2)

MyVec4{Int64}(1, 2)

In [19]:
v.x += 2

3

In [20]:
v

MyVec4{Int64}(3, 2)

**Major Downside** of mutability: mutable objects are usually allocated on the heap.

# Methods

In [21]:
function myplus(a::MyVec3, b::MyVec3)
    return MyVec3(a.x + b.x, a.y + b.y)
end

myplus (generic function with 1 method)

In [22]:
myplus(MyVec3(1,2), MyVec3(3,4))

MyVec3{Int64}(4, 6)

# Abstract Types

In [23]:
"""
An AbstractVec represents a 2-d vector.

New AbstractVec types should implement the methods horizontal(v) and vertical(v) to get the x and y coordinates
"""
abstract type AbstractVec end

AbstractVec

In [24]:
struct Vec{T} <: AbstractVec
    x::T
    y::T
end

horizontal(v::Vec) = v.x
vertical(v::Vec) = v.y

vertical (generic function with 1 method)

In [25]:
mutable struct MVec{T} <: AbstractVec
    x::T
    y::T
end

horizontal(v::MVec) = v.x
vertical(v::MVec) = v.y

vertical (generic function with 2 methods)

In [26]:
function myplus(a::AbstractVec, b::AbstractVec)
    return typeof(a)(horizontal(a) + horizontal(b), vertical(a) + vertical(b))
end

myplus (generic function with 2 methods)

In [27]:
myplus(Vec(1,2), MVec(3,4))

Vec{Int64}(4, 6)

In [28]:
struct ZeroVec <: AbstractVec end

horizontal(::ZeroVec) = 0.0
vertical(::ZeroVec) = 0.0

vertical (generic function with 3 methods)

In [29]:
myplus(Vec(1,2), ZeroVec())

Vec{Int64}(1, 2)

# Adding methods to functions from another module

In [30]:
@doc Base.zero

```
zero(x)
zero(::Type)
```

Get the additive identity element for the type of `x` (`x` can also specify the type itself).

See also [`iszero`](@ref), [`one`](@ref), [`oneunit`](@ref), [`oftype`](@ref).

# Examples

```jldoctest
julia> zero(1)
0

julia> zero(big"2.0")
0.0

julia> zero(rand(2,2))
2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
```


In [31]:
zero(Vec(1,2))

MethodError: MethodError: no method matching zero(::Vec{Int64})
The function `zero` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{LibGit2.GitHash})
   @ LibGit2 ~/.julia/juliaup/julia-1.11.4+0.x64.linux.gnu/share/julia/stdlib/v1.11/LibGit2/src/oid.jl:221
  zero(!Matched::Type{Pkg.Resolve.VersionWeight})
   @ Pkg ~/.julia/juliaup/julia-1.11.4+0.x64.linux.gnu/share/julia/stdlib/v1.11/Pkg/src/Resolve/versionweights.jl:15
  ...


In [34]:
Base.zero(v::Vec) = Vec(0,0)
Base.zero(v::MVec) = MVec(0,0)
Base.zero(v::ZeroVec) = ZeroVec()

In [35]:
zero(Vec(1,2))

Vec{Int64}(0, 0)

In [36]:
Base.:+(a::AbstractVec, b::AbstractVec) = typeof(a)(horizontal(a) + horizontal(b), vertical(a) + vertical(b))
Base.:+(a::ZeroVec, b::ZeroVec) = ZeroVec()

In [38]:
Vec(2, 1) + Vec(1, 2)
Vec(2, 1) + ZeroVec()

Vec{Int64}(2, 1)

In [39]:
function Base.:*(a::Int, b::AbstractVec)
    c = zero(b)
    for i in 1:a
        c += b
    end
    return c
end

In [42]:
1_000_000_000 * Vec(1, 1)

Vec{Int64}(1000000000, 1000000000)

In [43]:
2*MVec(1, 2)

MVec{Int64}(2, 4)

In [44]:
2*ZeroVec()

ZeroVec()

## Creating a Policy

In [ ]:
using POMDPs

In [ ]:
struct AlwaysRight <: Policy
end

In [ ]:
POMDPs.action(::AlwaysRight, state_or_belief) = :right